In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch import Tensor
from typing import Type, Any, Callable, Union, List, Optional
import os

In [2]:
# __all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
#            'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
#            'wide_resnet50_2', 'wide_resnet101_2']


# model_urls = {
#     'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
#     'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
#     'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
#     'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
#     'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
#     'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
#     'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
#     'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
#     'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
# }

In [3]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, k, kernel_size = 5, stride = 1):
        super(ConvBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels*k, kernel_size, stride)
        self.tanh = nn.Tanh()
        self.pool = nn.AvgPool2d(kernel_size = 2)
    
    def forward(self, x):
        x = self.conv(x)
        out = self.pool(self.tanh(x))
        
        return out    

In [4]:
class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        return logits

In [5]:
class Wide_LeNet4(nn.Module):
    def __init__(self, n_classes, k):
        super(Wide_LeNet4, self).__init__()
        self.k = k
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6*k, kernel_size=5, stride=1)
        self.pool1 = nn.AvgPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=6*k, out_channels=16*k, kernel_size=5, stride=1)
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(256, n_classes)
        
    def forward(self, x):
        x = self.pool1(self.tanh(self.conv1(x)))
        x = self.tanh(self.conv2(x))
        x = x.reshape(x.shape[0], -1)
        x = self.tanh(F.linear(x.shape[1], 256)(x))
        logits = self.fc2(x)
        return logits

In [6]:
print(Wide_LeNet4(10, 2))

Wide_LeNet4(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(12, 32, kernel_size=(5, 5), stride=(1, 1))
  (tanh): Tanh()
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


In [7]:
def wide_lenet4_2() -> Wide_LeNet4:
    # Wide LeNet4 with base_width k=2
    return Wide_LeNet4(10, 2)

def wide_lenet4_3():
    # Wide LeNet4 with base_width k=3
    return Wide_LeNet4(10,3)

In [8]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [9]:
best_train_acc = best_test_acc = start_epoch = 0

model = LeNet5(10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [10]:
def train(model, epoch):
    global best_train_acc, best_test_acc
    device = 'cuda'
    model.to(device)
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(f'Epoch {epoch} Step {batch_idx}/{len(trainloader)}', 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
        acc = 100.*correct/total
        if acc>best_train_acc:
            best_train_acc = acc

In [11]:
def test(model, epoch):
    global best_train_acc, best_test_acc
    device = 'cuda'
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print(f'Epoch {epoch} Step {batch_idx}/{len(testloader)}', 'Test Loss: %.3f | Test Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_test_acc:
        print('Saving..')
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, '/home/tr33/Downloads/checkpoint/ckpt.pth')
        best_test_acc = acc

In [12]:
best_acc = []
WideModel = wide_lenet4_2()
optimizer1 = optim.Adam(WideModel.parameters(), lr = 0.001)

In [13]:
# x = torch.ones((1, 3, 224, 224))
# print(WideModel(x.cuda()))

In [14]:
# Training ResNet34
for epoch in range(start_epoch, start_epoch+10):
    train(model, epoch)
    test(model, epoch)

best_acc.append([best_train_acc, best_test_acc])


Epoch: 0
Epoch 0 Step 0/1563 Loss: 2.311 | Acc: 6.250% (2/32)
Epoch 0 Step 1/1563 Loss: 2.306 | Acc: 6.250% (4/64)
Epoch 0 Step 2/1563 Loss: 2.301 | Acc: 10.417% (10/96)
Epoch 0 Step 3/1563 Loss: 2.306 | Acc: 10.156% (13/128)
Epoch 0 Step 4/1563 Loss: 2.302 | Acc: 10.000% (16/160)
Epoch 0 Step 5/1563 Loss: 2.301 | Acc: 9.896% (19/192)
Epoch 0 Step 6/1563 Loss: 2.305 | Acc: 9.821% (22/224)
Epoch 0 Step 7/1563 Loss: 2.302 | Acc: 10.547% (27/256)
Epoch 0 Step 8/1563 Loss: 2.298 | Acc: 11.458% (33/288)
Epoch 0 Step 9/1563 Loss: 2.292 | Acc: 12.812% (41/320)
Epoch 0 Step 10/1563 Loss: 2.290 | Acc: 13.352% (47/352)
Epoch 0 Step 11/1563 Loss: 2.290 | Acc: 13.802% (53/384)
Epoch 0 Step 12/1563 Loss: 2.287 | Acc: 13.702% (57/416)
Epoch 0 Step 13/1563 Loss: 2.278 | Acc: 14.286% (64/448)
Epoch 0 Step 14/1563 Loss: 2.268 | Acc: 15.208% (73/480)
Epoch 0 Step 15/1563 Loss: 2.258 | Acc: 16.602% (85/512)
Epoch 0 Step 16/1563 Loss: 2.259 | Acc: 16.360% (89/544)
Epoch 0 Step 17/1563 Loss: 2.257 | Acc: 

Epoch 0 Step 149/1563 Loss: 2.060 | Acc: 23.354% (1121/4800)
Epoch 0 Step 150/1563 Loss: 2.060 | Acc: 23.344% (1128/4832)
Epoch 0 Step 151/1563 Loss: 2.059 | Acc: 23.376% (1137/4864)
Epoch 0 Step 152/1563 Loss: 2.059 | Acc: 23.386% (1145/4896)
Epoch 0 Step 153/1563 Loss: 2.058 | Acc: 23.458% (1156/4928)
Epoch 0 Step 154/1563 Loss: 2.058 | Acc: 23.367% (1159/4960)
Epoch 0 Step 155/1563 Loss: 2.057 | Acc: 23.417% (1169/4992)
Epoch 0 Step 156/1563 Loss: 2.056 | Acc: 23.428% (1177/5024)
Epoch 0 Step 157/1563 Loss: 2.055 | Acc: 23.477% (1187/5056)
Epoch 0 Step 158/1563 Loss: 2.055 | Acc: 23.526% (1197/5088)
Epoch 0 Step 159/1563 Loss: 2.055 | Acc: 23.516% (1204/5120)
Epoch 0 Step 160/1563 Loss: 2.052 | Acc: 23.641% (1218/5152)
Epoch 0 Step 161/1563 Loss: 2.052 | Acc: 23.669% (1227/5184)
Epoch 0 Step 162/1563 Loss: 2.051 | Acc: 23.754% (1239/5216)
Epoch 0 Step 163/1563 Loss: 2.051 | Acc: 23.780% (1248/5248)
Epoch 0 Step 164/1563 Loss: 2.050 | Acc: 23.750% (1254/5280)
Epoch 0 Step 165/1563 Lo

Epoch 0 Step 288/1563 Loss: 1.997 | Acc: 26.092% (2413/9248)
Epoch 0 Step 289/1563 Loss: 1.997 | Acc: 26.121% (2424/9280)
Epoch 0 Step 290/1563 Loss: 1.997 | Acc: 26.117% (2432/9312)
Epoch 0 Step 291/1563 Loss: 1.997 | Acc: 26.134% (2442/9344)
Epoch 0 Step 292/1563 Loss: 1.997 | Acc: 26.173% (2454/9376)
Epoch 0 Step 293/1563 Loss: 1.997 | Acc: 26.180% (2463/9408)
Epoch 0 Step 294/1563 Loss: 1.997 | Acc: 26.229% (2476/9440)
Epoch 0 Step 295/1563 Loss: 1.997 | Acc: 26.267% (2488/9472)
Epoch 0 Step 296/1563 Loss: 1.996 | Acc: 26.294% (2499/9504)
Epoch 0 Step 297/1563 Loss: 1.995 | Acc: 26.321% (2510/9536)
Epoch 0 Step 298/1563 Loss: 1.995 | Acc: 26.359% (2522/9568)
Epoch 0 Step 299/1563 Loss: 1.995 | Acc: 26.375% (2532/9600)
Epoch 0 Step 300/1563 Loss: 1.996 | Acc: 26.360% (2539/9632)
Epoch 0 Step 301/1563 Loss: 1.995 | Acc: 26.397% (2551/9664)
Epoch 0 Step 302/1563 Loss: 1.994 | Acc: 26.454% (2565/9696)
Epoch 0 Step 303/1563 Loss: 1.994 | Acc: 26.460% (2574/9728)
Epoch 0 Step 304/1563 Lo

Epoch 0 Step 428/1563 Loss: 1.967 | Acc: 27.739% (3808/13728)
Epoch 0 Step 429/1563 Loss: 1.967 | Acc: 27.747% (3818/13760)
Epoch 0 Step 430/1563 Loss: 1.967 | Acc: 27.748% (3827/13792)
Epoch 0 Step 431/1563 Loss: 1.967 | Acc: 27.749% (3836/13824)
Epoch 0 Step 432/1563 Loss: 1.966 | Acc: 27.779% (3849/13856)
Epoch 0 Step 433/1563 Loss: 1.966 | Acc: 27.794% (3860/13888)
Epoch 0 Step 434/1563 Loss: 1.966 | Acc: 27.816% (3872/13920)
Epoch 0 Step 435/1563 Loss: 1.966 | Acc: 27.824% (3882/13952)
Epoch 0 Step 436/1563 Loss: 1.966 | Acc: 27.832% (3892/13984)
Epoch 0 Step 437/1563 Loss: 1.967 | Acc: 27.825% (3900/14016)
Epoch 0 Step 438/1563 Loss: 1.967 | Acc: 27.833% (3910/14048)
Epoch 0 Step 439/1563 Loss: 1.967 | Acc: 27.848% (3921/14080)
Epoch 0 Step 440/1563 Loss: 1.967 | Acc: 27.842% (3929/14112)
Epoch 0 Step 441/1563 Loss: 1.967 | Acc: 27.828% (3936/14144)
Epoch 0 Step 442/1563 Loss: 1.966 | Acc: 27.857% (3949/14176)
Epoch 0 Step 443/1563 Loss: 1.966 | Acc: 27.872% (3960/14208)
Epoch 0 

Epoch 0 Step 567/1563 Loss: 1.940 | Acc: 29.236% (5314/18176)
Epoch 0 Step 568/1563 Loss: 1.940 | Acc: 29.256% (5327/18208)
Epoch 0 Step 569/1563 Loss: 1.940 | Acc: 29.276% (5340/18240)
Epoch 0 Step 570/1563 Loss: 1.940 | Acc: 29.302% (5354/18272)
Epoch 0 Step 571/1563 Loss: 1.939 | Acc: 29.300% (5363/18304)
Epoch 0 Step 572/1563 Loss: 1.939 | Acc: 29.303% (5373/18336)
Epoch 0 Step 573/1563 Loss: 1.939 | Acc: 29.296% (5381/18368)
Epoch 0 Step 574/1563 Loss: 1.939 | Acc: 29.288% (5389/18400)
Epoch 0 Step 575/1563 Loss: 1.939 | Acc: 29.302% (5401/18432)
Epoch 0 Step 576/1563 Loss: 1.939 | Acc: 29.289% (5408/18464)
Epoch 0 Step 577/1563 Loss: 1.939 | Acc: 29.266% (5413/18496)
Epoch 0 Step 578/1563 Loss: 1.939 | Acc: 29.280% (5425/18528)
Epoch 0 Step 579/1563 Loss: 1.938 | Acc: 29.316% (5441/18560)
Epoch 0 Step 580/1563 Loss: 1.938 | Acc: 29.319% (5451/18592)
Epoch 0 Step 581/1563 Loss: 1.938 | Acc: 29.333% (5463/18624)
Epoch 0 Step 582/1563 Loss: 1.937 | Acc: 29.352% (5476/18656)
Epoch 0 

...

Epoch 9 Step 1291/1563 Loss: 1.348 | Acc: 51.475% (21282/41344)
Epoch 9 Step 1292/1563 Loss: 1.348 | Acc: 51.472% (21297/41376)
Epoch 9 Step 1293/1563 Loss: 1.348 | Acc: 51.463% (21310/41408)
Epoch 9 Step 1294/1563 Loss: 1.348 | Acc: 51.462% (21326/41440)
Epoch 9 Step 1295/1563 Loss: 1.349 | Acc: 51.454% (21339/41472)
Epoch 9 Step 1296/1563 Loss: 1.348 | Acc: 51.460% (21358/41504)
Epoch 9 Step 1297/1563 Loss: 1.348 | Acc: 51.457% (21373/41536)
Epoch 9 Step 1298/1563 Loss: 1.348 | Acc: 51.451% (21387/41568)
Epoch 9 Step 1299/1563 Loss: 1.349 | Acc: 51.452% (21404/41600)
Epoch 9 Step 1300/1563 Loss: 1.349 | Acc: 51.448% (21419/41632)
Epoch 9 Step 1301/1563 Loss: 1.349 | Acc: 51.445% (21434/41664)
Epoch 9 Step 1302/1563 Loss: 1.349 | Acc: 51.446% (21451/41696)
Epoch 9 Step 1303/1563 Loss: 1.349 | Acc: 51.450% (21469/41728)
Epoch 9 Step 1304/1563 Loss: 1.349 | Acc: 51.456% (21488/41760)
Epoch 9 Step 1305/1563 Loss: 1.349 | Acc: 51.467% (21509/41792)
Epoch 9 Step 1306/1563 Loss: 1.348 | Acc

Epoch 9 Step 1429/1563 Loss: 1.349 | Acc: 51.525% (23578/45760)
Epoch 9 Step 1430/1563 Loss: 1.349 | Acc: 51.522% (23593/45792)
Epoch 9 Step 1431/1563 Loss: 1.349 | Acc: 51.541% (23618/45824)
Epoch 9 Step 1432/1563 Loss: 1.349 | Acc: 51.546% (23637/45856)
Epoch 9 Step 1433/1563 Loss: 1.349 | Acc: 51.554% (23657/45888)
Epoch 9 Step 1434/1563 Loss: 1.349 | Acc: 51.559% (23676/45920)
Epoch 9 Step 1435/1563 Loss: 1.349 | Acc: 51.558% (23692/45952)
Epoch 9 Step 1436/1563 Loss: 1.349 | Acc: 51.559% (23709/45984)
Epoch 9 Step 1437/1563 Loss: 1.349 | Acc: 51.554% (23723/46016)
Epoch 9 Step 1438/1563 Loss: 1.348 | Acc: 51.566% (23745/46048)
Epoch 9 Step 1439/1563 Loss: 1.348 | Acc: 51.573% (23765/46080)
Epoch 9 Step 1440/1563 Loss: 1.348 | Acc: 51.572% (23781/46112)
Epoch 9 Step 1441/1563 Loss: 1.348 | Acc: 51.582% (23802/46144)
Epoch 9 Step 1442/1563 Loss: 1.348 | Acc: 51.577% (23816/46176)
Epoch 9 Step 1443/1563 Loss: 1.348 | Acc: 51.573% (23831/46208)
Epoch 9 Step 1444/1563 Loss: 1.348 | Acc

Epoch 9 Step 0/313 Test Loss: 1.018 | Test Acc: 59.375% (19/32)
Epoch 9 Step 1/313 Test Loss: 1.181 | Test Acc: 54.688% (35/64)
Epoch 9 Step 2/313 Test Loss: 1.198 | Test Acc: 58.333% (56/96)
Epoch 9 Step 3/313 Test Loss: 1.182 | Test Acc: 58.594% (75/128)
Epoch 9 Step 4/313 Test Loss: 1.204 | Test Acc: 58.125% (93/160)
Epoch 9 Step 5/313 Test Loss: 1.221 | Test Acc: 56.771% (109/192)
Epoch 9 Step 6/313 Test Loss: 1.287 | Test Acc: 54.018% (121/224)
Epoch 9 Step 7/313 Test Loss: 1.281 | Test Acc: 55.078% (141/256)
Epoch 9 Step 8/313 Test Loss: 1.276 | Test Acc: 54.861% (158/288)
Epoch 9 Step 9/313 Test Loss: 1.266 | Test Acc: 54.688% (175/320)
Epoch 9 Step 10/313 Test Loss: 1.263 | Test Acc: 54.261% (191/352)
Epoch 9 Step 11/313 Test Loss: 1.282 | Test Acc: 52.865% (203/384)
Epoch 9 Step 12/313 Test Loss: 1.269 | Test Acc: 53.606% (223/416)
Epoch 9 Step 13/313 Test Loss: 1.286 | Test Acc: 53.795% (241/448)
Epoch 9 Step 14/313 Test Loss: 1.292 | Test Acc: 53.542% (257/480)
Epoch 9 Step 

Epoch 9 Step 148/313 Test Loss: 1.266 | Test Acc: 55.055% (2625/4768)
Epoch 9 Step 149/313 Test Loss: 1.266 | Test Acc: 55.042% (2642/4800)
Epoch 9 Step 150/313 Test Loss: 1.268 | Test Acc: 54.967% (2656/4832)
Epoch 9 Step 151/313 Test Loss: 1.265 | Test Acc: 55.099% (2680/4864)
Epoch 9 Step 152/313 Test Loss: 1.266 | Test Acc: 55.065% (2696/4896)
Epoch 9 Step 153/313 Test Loss: 1.266 | Test Acc: 55.134% (2717/4928)
Epoch 9 Step 154/313 Test Loss: 1.266 | Test Acc: 55.161% (2736/4960)
Epoch 9 Step 155/313 Test Loss: 1.266 | Test Acc: 55.128% (2752/4992)
Epoch 9 Step 156/313 Test Loss: 1.267 | Test Acc: 55.115% (2769/5024)
Epoch 9 Step 157/313 Test Loss: 1.266 | Test Acc: 55.162% (2789/5056)
Epoch 9 Step 158/313 Test Loss: 1.267 | Test Acc: 55.031% (2800/5088)
Epoch 9 Step 159/313 Test Loss: 1.270 | Test Acc: 54.980% (2815/5120)
Epoch 9 Step 160/313 Test Loss: 1.270 | Test Acc: 55.047% (2836/5152)
Epoch 9 Step 161/313 Test Loss: 1.269 | Test Acc: 55.073% (2855/5184)
Epoch 9 Step 162/313

Epoch 9 Step 293/313 Test Loss: 1.279 | Test Acc: 54.688% (5145/9408)
Epoch 9 Step 294/313 Test Loss: 1.279 | Test Acc: 54.693% (5163/9440)
Epoch 9 Step 295/313 Test Loss: 1.279 | Test Acc: 54.709% (5182/9472)
Epoch 9 Step 296/313 Test Loss: 1.278 | Test Acc: 54.745% (5203/9504)
Epoch 9 Step 297/313 Test Loss: 1.279 | Test Acc: 54.719% (5218/9536)
Epoch 9 Step 298/313 Test Loss: 1.279 | Test Acc: 54.766% (5240/9568)
Epoch 9 Step 299/313 Test Loss: 1.278 | Test Acc: 54.792% (5260/9600)
Epoch 9 Step 300/313 Test Loss: 1.278 | Test Acc: 54.797% (5278/9632)
Epoch 9 Step 301/313 Test Loss: 1.278 | Test Acc: 54.801% (5296/9664)
Epoch 9 Step 302/313 Test Loss: 1.278 | Test Acc: 54.816% (5315/9696)
Epoch 9 Step 303/313 Test Loss: 1.278 | Test Acc: 54.842% (5335/9728)
Epoch 9 Step 304/313 Test Loss: 1.278 | Test Acc: 54.816% (5350/9760)
Epoch 9 Step 305/313 Test Loss: 1.277 | Test Acc: 54.820% (5368/9792)
Epoch 9 Step 306/313 Test Loss: 1.278 | Test Acc: 54.805% (5384/9824)
Epoch 9 Step 307/313

In [15]:
# Training Wide_ResNet16
best_train_acc = best_test_acc = 0
for epoch in range(start_epoch, start_epoch+10):
  train(WideModel, epoch)
  test(WideModel, epoch)
best_acc.append([best_train_acc, best_test_acc])


Epoch: 0


AttributeError: 'int' object has no attribute 'dim'

In [16]:
# !pip install torchsummary

In [17]:
from torchsummary import summary

In [18]:
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
              Tanh-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              Tanh-5           [-1, 16, 10, 10]               0
         AvgPool2d-6             [-1, 16, 5, 5]               0
            Conv2d-7            [-1, 120, 1, 1]          48,120
              Tanh-8            [-1, 120, 1, 1]               0
            Linear-9                   [-1, 84]          10,164
             Tanh-10                   [-1, 84]               0
           Linear-11                   [-1, 10]             850
Total params: 62,006
Trainable params: 62,006
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/ba

In [19]:
# summary(WideModel, (3,32,32))

In [20]:
print(best_acc)

[[65.625, 54.76]]
